## Предсказание стоимости жилья

В рамках этого проекта, моей задачей было обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Для оценки качества модели необходимо использовать метрики RMSE, MAE и R2.

In [ ]:
import pandas as pd
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics


pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator

RANDOM_SEED = 12345

# Подготовка данных

In [ ]:
# Инициализируем Спарк сессию
spark = SparkSession.builder \
                    .master("local") \
                    .appName("EDA California Housing") \
                    .getOrCreate()

In [ ]:
# Считаем необходимые данные и выведем их на экран

data = spark.read.load('/datasets/housing.csv',
                                            format="csv", sep=",", inferSchema=True, header="true")
data.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [ ]:
# Изучим тип данных в таблице
data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [ ]:
#Изучим пропуски в данных
columns = data.columns
for column in columns:
    print(column, data.filter(F.col(column).isNull()).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


Заполненим обнаруженные пропуски на основе данных в таблице

In [ ]:
#Значения с пропусками в столце total_bedrooms заполним нулем, так как это может означать что в этих массивах нет спален, или же пропуск может иметь другую природу
data = data.na.fill(0, subset='total_bedrooms')

columns = data.columns
for column in columns:
    print(column, data.filter(F.col(column).isNull()).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 0
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


In [ ]:
# Подробнее, изучим данные
data.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [ ]:
#Разделим колонки на числовые и категориальные
categorical_cols = ['ocean_proximity']
numerical_cols  = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']
target = ['median_house_value']

#Прокодируем категориальные переменные техникой OHE

In [ ]:
# Разделим выборки на тренировочную и тестовую
train_data, test_data = data.randomSplit([.8,.2], seed=RANDOM_SEED)
print(train_data.count(), test_data.count())

16431 4209


In [ ]:
# Прокодируем категориальные переменные техникой OHE
indexer = StringIndexer(inputCols=categorical_cols,
                        outputCols=[c+'_idx' for c in categorical_cols])
model = indexer.fit(train_data)
train_data = model.transform(train_data)
test_data = model.transform(test_data)

cols = [c for c in data.columns for i in categorical_cols if (c.startswith(i))]

encoder = OneHotEncoder(inputCols = [c+'_idx' for c in categorical_cols],
                        outputCols= [c+'_ohe' for c in categorical_cols])

model = encoder.fit(train_data)
train_data = model.transform(train_data)
test_data = model.transform(test_data)

cols = [c for c in train_data.columns for i in categorical_cols if (c.startswith(i))]

categorical_assembler = \
        VectorAssembler(inputCols=[c+'_ohe' for c in categorical_cols],
                                        outputCol="categorical_features")
train_data = categorical_assembler.transform(train_data)
test_data = categorical_assembler.transform(test_data)

train_data.show(3)
test_data.show(3)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|categorical_features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|           94600.0|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|       (4,[2],[1.0])|
|   -124.3|    41.8|              19.0|     2672.0|         552.0|    1298.0|     478.0|       1.9797|           85800.0|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|       (4,[2],[1.0])|
|   -

In [ ]:
# Также проведем и трансформацию числовых переменных
numerical_assembler = VectorAssembler(inputCols=numerical_cols,
                                                                            outputCol="numerical_features")
train_data = numerical_assembler.transform(train_data)
test_data = numerical_assembler.transform(test_data)


standardScaler = StandardScaler(inputCol='numerical_features',
                                                                outputCol="numerical_features_scaled")
model2 = standardScaler.fit(train_data)

train_data = model2.transform(train_data)
test_data = model2.transform(test_data)


#И соединим полученные данные в единый массив для обучения модели
all_features = ['categorical_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features,
                                  outputCol="features")

train_data = final_assembler.transform(train_data)
test_data = final_assembler.transform(test_data)

train_data.select(all_features).show(3)
test_data.select(all_features).show(3)

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[2],[1.0])|     [-62.004132451563...|
|       (4,[2],[1.0])|     [-61.979201155845...|
|       (4,[2],[1.0])|     [-61.979201155845...|
+--------------------+-------------------------+
only showing top 3 rows

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[2],[1.0])|     [-61.944297341839...|
|       (4,[2],[1.0])|     [-61.914379786977...|
|       (4,[2],[1.0])|     [-61.914379786977...|
+--------------------+-------------------------+
only showing top 3 rows



# Обучение моделей

1. Проведем обучение и тест на модели с использвоанием всех переменных из датасета

In [ ]:
# Разьделим выборки на тренировочную и тестовую
train_data, test_data = data.randomSplit([.8,.2], seed=RANDOM_SEED)

In [ ]:
lr = LinearRegression(featuresCol = 'features', labelCol='median_house_value')

model = lr.fit(train_data)

predictions = model.transform(test_data)

predictedLabes = predictions.select("median_house_value", "prediction")
predictedLabes.show()

23/05/24 08:19:10 WARN Instrumentation: [593657e2] regParam is zero, which might cause numerical instability and overfitting.
23/05/24 08:19:11 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/05/24 08:19:11 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/05/24 08:19:12 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/05/24 08:19:12 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          106700.0|217401.41246218001|
|          128900.0|207761.73643585527|
|          116100.0|234718.96505517233|
|           70500.0|164513.37888022605|
|           85600.0|188814.00482306024|
|           75500.0|138938.81329614995|
|           79600.0|160958.21959933173|
|           92800.0| 210492.6480937996|
|           97300.0|167326.85464484943|
|           82100.0|159888.40428816061|
|          126900.0|158732.02491717553|
|          119400.0|164132.65479839547|
|           71300.0|168399.54506612103|
|           75600.0| 149361.3097679494|
|           98800.0|152004.54652441572|
|           92600.0|150933.18413452618|
|          152700.0| 142109.9946842366|
|          150000.0|156604.09873819677|
|           74000.0|156448.52670405107|
|           82400.0|162852.33443435607|
+------------------+------------------+
only showing top 20 rows



In [ ]:
# И расчитаем метрики, на полученных данных
trainingSummary = model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
print("MAE: %f" % trainingSummary.meanAbsoluteError)

RMSE: 69052.301074
r2: 0.641052
MAE: 50203.809138


2. Теперь проведем обучение и тест на модели с использвоанием только числовых переменных из датасета, так как согласно информации от заказчика, есть вероятность, что категориальные переменные не оказывают на итоговоую стоимость и как следствие, не будут оказывать большого влияния на предсказание модели и могут только увеличивать время обучения.

In [ ]:
data2 = data
data2.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [ ]:
# Обучим модель, только на количественных данных
lr = LinearRegression(featuresCol = 'numerical_features_scaled', labelCol='median_house_value')

model = lr.fit(train_data)

predictions = model.transform(test_data)

predictedLabes = predictions.select("median_house_value", "prediction")
predictedLabes.show()

23/05/24 08:21:35 WARN Instrumentation: [6a8be4bf] regParam is zero, which might cause numerical instability and overfitting.


+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          106700.0|190476.72406287305|
|          128900.0|174503.83135654917|
|          116100.0| 201586.9150031726|
|           70500.0|131651.32028728584|
|           85600.0|156120.77340789698|
|           75500.0| 100616.3021771675|
|           79600.0|130568.97371992283|
|           92800.0| 177300.8879561834|
|           97300.0| 141236.2185783363|
|           82100.0|123105.43772208318|
|          126900.0|120256.44045340223|
|          119400.0|122973.01408903999|
|           71300.0|144094.87834225176|
|           75600.0|116970.44949270599|
|           98800.0|117447.50912469765|
|           92600.0|122847.52569746552|
|          152700.0| 93984.27938967152|
|          150000.0|146076.57979437802|
|           74000.0|131309.15814570012|
|           82400.0|140310.33897198178|
+------------------+------------------+
only showing top 20 rows



In [ ]:
#И изучим значение метрики
trainingSummary = model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
print("MAE: %f" % trainingSummary.meanAbsoluteError)

RMSE: 69983.016885
r2: 0.631311
MAE: 51306.398891


In [ ]:
# Подсчитаем разницу между указанными значениями, а именно разницу между обучением без категориальных переменных  , по сравнению с результатом обучения с категориальными переменными
# Результаты обучения без категориальных переменных - результаты обучения с категориальными переменными
RMSE = 69983.016885 - 69052.301074
r2 = 0.631311 - 0.641052
MAE = 51306.398891 - 50203.809138
print(RMSE)
print(r2)
print(MAE)

930.7158110000018
-0.009741
1102.5897530000002


Вспоминаем, как эти метрики, показывают качество можели:

- RMSE - чем ниже, тем лучше

- R2 - чем больше, тем лучше

- MAE - чем ниже, тем лучше

In [ ]:
# По окончанию работы, не забываем остановить сессию
spark.stop()

# Анализ результатов

Результат модели обученной на датасете без категориальных переменных привели лишь к ухудшению качества предсказания модели по всем метрикам, которыми мы оценивали ее качество, из чего мы можем сделать вывод, что использование категориальных переменных в обучение модели, в данном датасете влияет и оказывает немаловажное влияние на итоговое качество модели.